In [4]:
from scipy.stats import beta
import scipy.integrate as integrate
import numpy as np
import pandas as pd

from scipy import optimize

In [ ]:
n_cities = 200
sequence = np.linspace(1/n_cities/2, 1-1/n_cities/2, num=n_cities)
D = 0.2

def mean_abs_deviation(a, b):
    x = np.linspace(1/16_000, 1-1/16_000, 16_000)
    return integrate.simpson(y=np.abs(x - beta.median(a, b))*beta.pdf(x, a, b), x=x)

def functions(params, median_value):
    a, b = params
    return (mean_abs_deviation(a, b)-D)**2+(beta.median(a, b)-median_value)**2

def calucate(m, start_point):
    opt = optimize.minimize(functions, start_point, tol=1e-11, method='Nelder-Mead', args=m, options={'maxiter': 1e+4})
    if opt['success']:return opt['x']
    else: raise RuntimeError(f'{opt['message']}')

In [6]:
L = []
a, b = 0.5, 0.5
for m in sequence[n_cities//2:]:
    a, b = calucate(m, (a, b))
    L.append((a, b))

In [15]:
second_part = np.array(L)
first_part = np.flip(second_part)
df = pd.DataFrame(np.concatenate((first_part, second_part)), columns=['a', 'b'])
df['median'] = beta.median(df.a, df.b)
df['diff']  = abs(df['median']-np.array(sequence))
df['true_value'] = sequence

In [17]:
df.to_parquet(f'parametry_beta_D{int(100*D)}_n{n_cities}.parquet', index=False)